In [1]:
import os
AVAILABLE_GPUS = '2,3'
os.environ['CUDA_VISIBLE_DEVICES'] = AVAILABLE_GPUS

import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, GPTNeoXForCausalLM

for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

NVIDIA RTX A6000
NVIDIA RTX A6000


In [2]:
model_name_or_path = "OpenAssistant/pythia-12b-sft-v8-rlhf-2k-steps" #23GB
number_model_per_gpu = 2
number_of_gpus = len(AVAILABLE_GPUS.split(','))
print("Number of GPUs:", number_of_gpus)
model_instances = []
for i in range(number_of_gpus):
    for j in range(number_model_per_gpu):
        #isntanciate the model on the right GPU
        model_instances.append(AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16).todevice(i))
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side="left", legacy=False)

Number of GPUs: 2


In [38]:
import pandas as pd
import json

def load_from_jsonl(file_name: str) -> list[dict]:
    def load_json_line(line: str, i: int, file_name: str):
        try:
            return json.loads(line)
        except:
            raise ValueError(f"Error in line {i+1}\n{line} of {file_name}")
    with open(file_name, "r") as f:
        data = [load_json_line(line, i, file_name) for i, line in enumerate(f)]
    return data

def get_prompt(d):
    prompt = d['prompt'][0]['content']
    OASS_prompt = '<|prompter|>' + prompt + '<|endoftext|><|assistant|>Here are my thoughts on the poem:'
    return tokenizer.encode(OASS_prompt, return_tensors='pt')

dataset_filename: str = "feedback_poem.jsonl"
dataset = load_from_jsonl(dataset_filename)
pure_dataset = [get_prompt(d) for d in dataset]

['Here', 'Ġare', 'Ġmy', 'Ġthoughts', 'Ġon', 'Ġthe', 'Ġpoem', ':', 'test', 'est', 'est']


In [40]:
from datasets import Dataset
from torch.utils.data import DataLoader

ds = Dataset.from_dict({"data": pure_dataset})
print(ds[0])
print(type(ds[0]))
print(len(ds))
dataloader = DataLoader(ds, batch_size=10)
for batch in dataloader:
    print(batch)

{'data': [[50279, 7845, 4385, 13366, 327, 253, 1563, 19361, 15, 187, 8816, 358, 27, 346, 688, 253, 19929, 273, 23147, 396, 5047, 532, 414, 13, 187, 2811, 10763, 285, 13742, 345, 13956, 13, 187, 66, 13341, 46526, 273, 17155, 40994, 1445, 4721, 5369, 3502, 13, 187, 3113, 17059, 273, 439, 22369, 272, 331, 472, 461, 187, 680, 1261, 37619, 1523, 715, 253, 8067, 273, 6242, 15, 187, 4678, 1920, 90, 13, 751, 247, 7771, 8211, 38985, 13, 187, 69, 1972, 2220, 253, 479, 16044, 273, 776, 23729, 13, 187, 1258, 343, 1076, 342, 776, 21768, 13, 187, 2484, 261, 35025, 20196, 273, 11010, 285, 4282, 15, 187, 18291, 13, 347, 359, 3986, 562, 281, 30231, 697, 1162, 1568, 267, 15926, 13, 187, 262, 42282, 1977, 13, 6108, 441, 6345, 1417, 13, 187, 2913, 920, 323, 697, 2563, 47842, 267, 5181, 15, 187, 1898, 13742, 13, 326, 48775, 21791, 336, 560, 13, 187, 42241, 617, 1133, 2220, 1495, 434, 29381, 19103, 13, 187, 16056, 920, 247, 4384, 273, 29444, 1430, 15, 187, 2993, 994, 88, 1100, 11865, 13, 751, 479, 48299, 21

RuntimeError: each element in list of batch should be of equal size

In [26]:
print(ds[0])
print(len(ds))

{'prompt': 'Please comment briefly on the following poem.\nPoem: "In the realm of cosmic serendipity,\nwhere beauty and fate conjoin,\na tapestry of celestial artistry unfolds,\nwith threads of shimmering stardust\nwoven delicately into the fabric of existence.\nBeauty, like a fleeting butterfly,\ndances upon the meadows of our souls,\nflirting with our senses,\nwhispering secrets of joy and wonder.\nYet, as we reach out to clutch its ethereal wings,\nit slips away, leaving us breathless,\nyearning for its ephemeral touch.\nAnd fate, that capricious sorceress,\nplays her hand upon life\'s intricacies,\nspinning a web of unpredictability.\nShe entwines paths, like meandering rivers,\nconverging and diverging through time,\nleading us towards destinations unknown.\nBut behold, dear seeker of truth,\nfor in the depths of fate\'s enigmatic realm,\nbeauty finds solace, cradled within destiny\'s arms.\nFor no matter the twists and turns,\nthe valleys and mountains we encounter,\nbeauty is an

In [18]:
print(len(pure_dataset))
print(type(pure_dataset))
print(pure_dataset[0])

2000
<class 'list'>
Please comment briefly on the following poem.
Poem: "In the realm of cosmic serendipity,
where beauty and fate conjoin,
a tapestry of celestial artistry unfolds,
with threads of shimmering stardust
woven delicately into the fabric of existence.
Beauty, like a fleeting butterfly,
dances upon the meadows of our souls,
flirting with our senses,
whispering secrets of joy and wonder.
Yet, as we reach out to clutch its ethereal wings,
it slips away, leaving us breathless,
yearning for its ephemeral touch.
And fate, that capricious sorceress,
plays her hand upon life's intricacies,
spinning a web of unpredictability.
She entwines paths, like meandering rivers,
converging and diverging through time,
leading us towards destinations unknown.
But behold, dear seeker of truth,
for in the depths of fate's enigmatic realm,
beauty finds solace, cradled within destiny's arms.
For no matter the twists and turns,
the valleys and mountains we encounter,
beauty is an eternal companion,

In [14]:
print(type(dataset))
print(len(dataset))
print(dataset[0])
print(dataset[0].keys())
print(dataset[0]['prompt'][0].keys())
print(dataset[1]['prompt'][0]['content'])

<class 'list'>
2000
{'prompt': [{'type': 'human', 'content': 'Please comment briefly on the following poem.\nPoem: "In the realm of cosmic serendipity,\nwhere beauty and fate conjoin,\na tapestry of celestial artistry unfolds,\nwith threads of shimmering stardust\nwoven delicately into the fabric of existence.\nBeauty, like a fleeting butterfly,\ndances upon the meadows of our souls,\nflirting with our senses,\nwhispering secrets of joy and wonder.\nYet, as we reach out to clutch its ethereal wings,\nit slips away, leaving us breathless,\nyearning for its ephemeral touch.\nAnd fate, that capricious sorceress,\nplays her hand upon life\'s intricacies,\nspinning a web of unpredictability.\nShe entwines paths, like meandering rivers,\nconverging and diverging through time,\nleading us towards destinations unknown.\nBut behold, dear seeker of truth,\nfor in the depths of fate\'s enigmatic realm,\nbeauty finds solace, cradled within destiny\'s arms.\nFor no matter the twists and turns,\nthe

In [4]:
import jsonlines
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, file_path):
        self.data = []
        with jsonlines.open(file_path) as reader:
            for obj in reader:
                self.data.append(obj)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
ds = MyDataset('feedback_poem.jsonl')


ModuleNotFoundError: No module named 'jsonlines'

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
device = accelerator.device

model = accelerator.prepare_model(model)
data_loader = accelerator.prepare_data_loader(data_loader, device_placement=True)


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import multiprocessing
from multiprocessing import Process

# Function to load the model and run it on some prompts
def run_model_on_gpu(prompts, gpu_index):
    # Specify the particular GPU to use
    device = torch.device(f'cuda:{gpu_index}')
    
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
    
    for prompt in prompts:
        # Tokenize the prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        # Generate a response
        outputs = model.generate(**inputs, max_length=50)
        # Decode the response
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        print(f"GPU {gpu_index}: {text}")


if __name__ == "__main__":
    # List of prompts to be processed
    prompts_list = [
        "What is the capital of France?",
        "Who wrote 1984?",
        "Explain the theory of relativity",
        # ... more prompts
    ]
    
    # Assuming you have 4 GPUs available and you want to split prompts equally
    num_gpus = 4
    num_prompts_per_gpu = len(prompts_list) // num_gpus

    # Creating processes
    processes = []

    for i in range(num_gpus):
        start_index = i * num_prompts_per_gpu
        # Use the modulo operator to handle any leftover prompts
        end_index = (i + 1) * num_prompts_per_gpu if i < num_gpus - 1 else len(prompts_list)
        prompts_sublist = prompts_list[start_index:end_index]
        p = Process(target=run_model_on_gpu, args=(prompts_sublist, i))
        processes.append(p)

    # Start all the processes
    for p in processes:
        p.start()

    # Wait for all processes to finish
    for p in processes:
        p.join()
